In [2]:
import numpy as np
import os, sys
import pickle, functools, operator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random #shuffle
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from netCDF4 import Dataset

import numpy as np
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from netCDF4 import Dataset

2021-12-09 17:07:29.204148: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 
    
    sst_3 = np.zeros((myform,winnum,12,24,72))
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    #t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]


    #sst_4 = sst_4.swapaxes(1, 3)
    trX = sst_4
    del sst_1,sst_2,sst_3,sst_4
    #trX = trX.reshape(int(winnum*myform),12,1728)

    #保存np数组
    #(5859, 12, 24, 72, 2)
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

In [5]:
trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/liyanqiu/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/liyanqiu/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

testX, testY = GOSDAdata('/home/d/Q/liyanqiu/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/liyanqiu/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]

In [6]:
input_cnn = Input(shape=(24, 72,1))

x = layers.Conv2D(30, (8,4), activation='tanh', padding='SAME')(input_cnn)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.reshape(x, [-1, 30 * 18 * 6])
outputs_cnn = x
model_cnn = Model(input_cnn,outputs_cnn)


2021-12-09 17:07:52.505393: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-09 17:07:52.505860: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-09 17:07:52.538865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 17:07:52.539001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.845GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2021-12-09 17:07:52.539014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 17:07:52.539815: I tensorflow/stream_executor/platform/default/d

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: all CUDA-capable devices are busy or unavailable

In [19]:
x1 = layers.Conv2D(50, (8,4), activation='tanh', padding='SAME')(input_cnn)
x1 = tf.nn.max_pool(x1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x1 = layers.Conv2D(50, (8,4), activation='tanh', padding='SAME')(x1)
x1 = tf.nn.max_pool(x1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x1 = layers.Conv2D(50, (8,4), activation='tanh', padding='SAME')(x1)
x1 = tf.reshape(x1, [-1, 50 * 18 * 6])
outputs_cnn1 = x1
model_cnn1 = Model(input_cnn,outputs_cnn1)

In [5]:
_input = Input(shape=(12, 24, 72))
_input1 = tf.reshape(_input, [-1, 24, 72])
x = model_cnn(_input1)
#x1 = model_cnn1(_input1)
output = tf.reshape(x, [-1,12, 30 * 18 * 6])
#output1 = tf.reshape(x1, [-1,12, 50 * 18 * 6])
#output = tf.concat([output,output1],2)#(None, 12, 6480)

In [6]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


In [7]:
time_steps_encoder=12
#num_encoder_tokens=5400
num_encoder_tokens=5400
latent_dim=1024
time_steps_decoder=24
#time_steps_decoder=3
num_decoder_tokens=1

encoder_inputs = output
"""后为特征数，前为批次数量！！！"""
#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
# 我们设定我们的解码器回传整个输出的序列同时也回传内部的states参数
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
# 我们使用`encoder_states`来做为初始值(initial state) <-- 重要
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])


# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

decoder_dense = TimeDistributed(Dense(latent_dim))
decoder_outputs1 = decoder_dense(decoder_concat_input)

decoder_dense2 = TimeDistributed(Dense(num_decoder_tokens))
decoder_outputs2 = decoder_dense2(decoder_outputs1)

# 定义一个模型接收encoder_input_data` & `decoder_input_data`做为输入而输出`decoder_target_data`
model = Model([_input, decoder_inputs], decoder_outputs2)
model.summary()
#plot_model(model, to_file='model_train.png', show_shapes=True, show_layer_names=True)


epochs=100
# Early Stopping
earlystopping = EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights = True)

# Tensorboard callback
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Run training
#opt = keras.optimizers.Adam(lr = 0.0001)
#RMSprop
#当学习停滞时，减少2倍或10倍的学习率常常能获得较好的效果。
x = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,patience=2,verbose=0,mode="auto")

model.compile(metrics=['mae'], optimizer='Adam', loss='mse')

model.fit([trX, trY_decoder_input], trY, batch_size=32, epochs=epochs, validation_split=0.3, verbose=1, callbacks=[earlystopping, x])


TypeError: Exception encountered when calling layer "tf.keras.backend.rnn" (type TFOpLambda).

You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 12), dtype=tf.float32, name=None), name='tf.compat.v1.nn.softmax_1/Softmax:0', description="created by layer 'tf.compat.v1.nn.softmax_1'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

Call arguments received:
  • step_function=<function AttentionLayer.call.<locals>.energy_step at 0x7fc7844867a0>
  • inputs=tf.Tensor(shape=(None, 24, 1024), dtype=float32)
  • i

In [ ]:
# Encode the input sequence to get the feature vector
# 对输入序列进行编码得到特征向量
encoder_model = Model(_input,[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
# 下面的张量将保存上一个时间步的状态
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
#修改
decoder_hidden_state_input = Input(shape=(time_steps_encoder,latent_dim))

# Get the embeddings of the decoder sequence
# 获取解码器序列的嵌入
#dec_emb2= dec_emb_layer(decoder_inputs)
decoder_inputs = Input(shape=(None, num_decoder_tokens))#注意是none (none, 1)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
# 要预测序列中的下一个单词，请将初始状态设置为前一个时间步的状态
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_inputs, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
# 注意力 推理
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
# 用于生成概率分布的密集 softmax 层。 超过目标词汇
decoder_outputs22 = decoder_dense(decoder_inf_concat)
decoder_outputs33 = decoder_dense2(decoder_outputs22)

# Final decoder model
# 最终解码器模型
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs33] + [state_h2, state_c2])


#预测
def decode_predict(input_seq):
    # 将输入编码为状态向量。
    e_out, e_h, e_c = encoder_model.predict(input_seq.reshape(-1, 12, 24, 72))
    #print(input_seq.shape)
    # 生成长度为 1 的空目标序列。
    target_seq = np.zeros(( 1,1, num_decoder_tokens))#(1, 1, 1)
    
    # Populate the first word of target sequence with the start word.
    # 用起始词填充目标序列的第一个词。
    target_seq[0, 0,0] = 0.

    decoded_sentence = []
    for i in range(24):
        output, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        #print(output.shape)  (1,1,1)
        
        decoded_sentence.append(output[0,0,0])

        # Update the target sequence (of length 1).
        # 更新目标序列（长度为 1）。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0,0,0] = output
        #print(target_seq)

        # Update internal states
        # 更新内部状态
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
out = []
for i in range(testX.shape[0]):
    print("test No." + str(i+1))
    out.append(decode_predict(testX[i,:,:]))

In [17]:
cor = np.zeros((24))
for i in range(24):
    cor[i] = np.corrcoef(testY.reshape(384,24)[:,i],np.array(out)[:,i])[0,1]
print(cor)
print("sum :" + str(np.sum(cor)))

[0.97421843 0.94444138 0.90324016 0.85212566 0.79523569 0.74018026
 0.69168632 0.6516915  0.62270184 0.6054359  0.59965461 0.59770208
 0.58874442 0.56469969 0.52325229 0.4667135  0.39938852 0.32956909
 0.26729814 0.21822757 0.18278565 0.1589009  0.14596835 0.14196553]
sum :12.965827470791154


In [31]:
from sklearn.metrics import mean_squared_error, r2_score
print(f"均方误差(MSE)：{mean_squared_error(out[35], testY[35])}")

均方误差(MSE)：0.4820995492646536
